Testing spaCy for Capstone Project

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import spacy

In [2]:
nlp_sm = spacy.load('en_core_web_sm')
nlp_lg = spacy.load('en_core_web_lg')

In [3]:
data = pd.read_csv('./data/chewy.csv')

In [4]:
# If 'Unnamed: 0' column is in the data frame, run this cell 
data.drop(columns='Unnamed: 0', inplace=True)

In [14]:
# messing around on first review
tokens = nlp_sm(data['reviews'][0])

token_list = []
for token in tokens[:10]:
    token_list.append(token.text)

token_list 
#     print(type(token))
#     print(token.text, token.has_vector, token.vector_norm, token.is_oov)

['I', 'do', 'not', 'know', 'how', 'to', 'rate', 'this', '.', 'The']

In [6]:
# def spacy(index, df):
#     tokens = nlp_sm(df['reviews'][i])
#     for token in tokens:
#         type(token)
# #         token = str(token)
# #         df['token_text'][i] = token.text
#         df['token_has_vector'][i] = token.has_vector
#         df['token_vector_norm'][i] = token.vector_norm
#         df['token_is_oov'][i] = token.is_oov

In [7]:
# # need to create the columns I want to add before putting the values in them 
# # data['token_text'] = 0
# data['token_has_vector'] = 0
# data['token_vector_norm'] = 0
# data['token_is_oov'] = 0

In [8]:
# for i in data.index:
#     spacy(data.loc[i, 'reviews'], data)
# data.head(11)

In [9]:
# def spacy_function(review, dataframe):
#     tokens = nlp_sm(review)
#     tokens_text = []
#     tokens_has_vector = []
#     tokens_vector_norm = []
#     tokens_is_oov = []
    
#     for token in tokens:
#         text = []
#         has_vector = []
#         vector_norm = []
#         is_oov = []
        
#         text.append(token.text)
#         has_vector.append(token.has_vector)
#         vector_norms.append(token.vector_norm)
#         is_oov.append(token.is_oov)
        
#         tokens_text.append(text)
#         tokens_has_vectors.append(has_vectors)
#         tokens_vector_norms.append(vector_norms)
#         tokens_is_oov.append(is_oov)

In [10]:
# df['reviews'].apply(lambda x: spacy_function(x, df))

In [12]:
data.head()

,title,price,descriptions,key_benefits,rating,reviews,subcat,cat
0,Nylabone Teething Pacifier Puppy Chew Toy,3.59,'Every puppy needs a pacifier to soothe teethi...,Designed to encourage positive play and teach ...,4.2,I do not know how to rate this. The puppy this...,moderate,chew toys
1,"KONG Puppy Dog Toy, Color Varies",6.99,"""The Puppy KONG dog toy is customized for a gr...",Unpredictable bounce is great for energetic pu...,4.3,"I have had dozens of dogs over the years, and ...",moderate,chew toys
2,Petstages Dogwood Tough Dog Chew Toy,8.83,"""Chewing is a natural behavior in all dogs, as...",Chew toy that combines real wood with syntheti...,4.2,"Our 8 month old, shepherd/mastiff has had this...",moderate,chew toys
3,Nylabone Teething Rings Puppy Chew Toy,6.57,'Great for teething and tugging! The Puppy Tee...,Designed to encourage positive play and teach ...,4.1,I have a small shih tzu who absolutely loves t...,moderate,chew toys
4,Nylabone Puppy Teething X Bone Beef Flavored P...,6.89,'Curious puppies have met their match with the...,Non-edible dog toy is made for teething puppie...,4.1,This is a great shoe for a tiny puppy in there...,moderate,chew toys


This next section will be usefull code for the actual recommendation system 

In [17]:
# https://medium.com/@armandj.olivares/building-nlp-content-based-recommender-systems-b104a709c042

list_docs = []
for i in range(len(data)):
    if data['reviews'][i] != '':
        doc = nlp_lg("u'" + str(data['reviews'][i]) + "'")
        list_docs.append(doc)

In [28]:
# https://medium.com/@armandj.olivares/building-nlp-content-based-recommender-systems-b104a709c042
# This will take in what the users says and then score it based off that 
# I have added to this as well
def calculate_similarity_with_spacy(nlp, df, user_text, n=6):
    # Calculate similarity with Spacy 
    list_sim = []
    toy_score = []
    doc1 = nlp('u' + user_text + "'")
    for i in df.index:
        try: 
            doc2 = list_docs[i]
            score = doc1.similarity(doc2)
            list_sim.append((doc1, doc2, score))
            toy_score.append((score, df['title'][i]))
        except:
            continue
    return toy_score  #list_sim

In [30]:
ranking_list = calculate_similarity_with_spacy(nlp_lg, data, 'I want a toy for a dog that is energetic and a chewer.')

In [35]:
ranked_dict = {}
for i, score_toy in enumerate(ranking_list[:10]):
    print(i, score_toy[0])
    ranked_dict[score_toy[0]] = {i : score_toy[1]}

0 0.8802032025720885
1 0.884762871109849
2 0.8786090723382932
3 0.8906501516502305
4 0.8937388735600288
5 0.8868641654501501
6 0.8856358906355211
7 0.8783410177545526
8 0.8940829394859542
9 0.8745336489574868


In [46]:
# https://www.geeksforgeeks.org/python-sort-python-dictionaries-by-key-or-value/
final_dict = {}
for i in sorted(ranked_dict, reverse=True) : 
    print ((i, ranked_dict[i]), end ='\n')
#     final_dict[i] = ranked_dict[i]
# final_dict

(0.8940829394859542, {8: 'KONG Puppy Goodie Bone Dog Toy, Color Varies'})
(0.8937388735600288, {4: 'Nylabone Puppy Teething X Bone Beef Flavored Puppy Chew Toy, Small'})
(0.8906501516502305, {3: 'Nylabone Teething Rings Puppy Chew Toy'})
(0.8868641654501501, {5: 'Nylabone Puppy Petite Dental Puppy Chew Toy'})
(0.8856358906355211, {6: 'Nylabone Puppy Chicken Flavored Teething Dinosaur Puppy Chew Toy, Petite'})
(0.884762871109849, {1: 'KONG Puppy Dog Toy, Color Varies'})
(0.8802032025720885, {0: 'Nylabone Teething Pacifier Puppy Chew Toy'})
(0.8786090723382932, {2: 'Petstages Dogwood Tough Dog Chew Toy'})
(0.8783410177545526, {7: 'Petstages Cool Teething Stick Tough Dog Chew Toy'})
(0.8745336489574868, {9: "Hartz Chew 'n Clean Tuff Bone Tough Dog Chew Toy Toy, Color Varies"})
